# 네이버 뉴스 스크래핑

https://news.naver.com/

In [1]:
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-brower/chromdriver /usr/bin

# import sys
# sys.path.insert(0, '/usr/lib/chromium-brower/chromdriver')

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')



## `news_scraping()`: 뉴스 스크래핑 함수

In [3]:
def news_scraping(news_url, wd):
    press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
    title = wd.find_element_by_id('articleTitle').text
    date = wd.find_element_by_class_name('t11').text
    article = wd.find_element_by_id('articleBodyContents').text
    
    article = article.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}","")
    article = article.replace("function _flash_removeCallback() {}","")
    article = article.replace("\n","")
    article = article.replace("\t","")
    article = article.replace("<!-- 본문 내용 -->","")
    
    good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
    warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
    sad = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
    angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
    want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
    
    recom = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text
    
    print("뉴스:",[title, press, date, good, warm, sad, angry, want, recom])
    
    return [title, press, date, article, good, warm, sad, angry, want, recom]

## `comments_scraping()`: 뉴스 댓글 스크래핑 함수

In [ ]:
def comments_scraping(news_url, wd):
    # 우선 댓글 창을 열고, 모든 댓글을 연다.
    
    try:
        wd.find_element_by_class_name('u_cbox_in_view_comment').click()
        time.sleep(3)
        print("[댓글더보기]",end="")
        
        while True:
            
            wd.find_element_by_class_name('u_cbox_page_more').click()
            print("[더보기]", end="")
            time.sleep(1)
    except:
        pass
    
    btn_cnt_list = [i.text for i in wd.find_elements_by_class_name('u_cbox_reply_cnt')]
    time.sleep(1)
    btn_reply_list = wd.find_elements_by_class_name('u_cbox_btn_reply')
    time.sleep(1)

    for btn_cnt, btn_reply  in zip(btn_cnt_list, btn_reply_list):
        if int(btn_cnt) != 0:
            btn_reply.send_keys('\n')
            print("[답글]", end='')
            time.sleep(0.5)
    
    print("[댓글 스크래핑]", end='')
    comments_idx = 0 
    comments_df = pd.DataFrame(columns=("Contents", "Name", "Datetime",
                                        "Recommend","Unrecommend","URL"))
    
    comments = wd.find_elements_by_class_name('u_cbox_comment_box')
    for comment in comments:
        try:
            name = comment.find_element_by_class_name('u_cbox_nick').text
            date = comment.find_element_by_class_name('u_cbox_date').text
            contents = comment.find_element_by_class_name('u_cbox_contents').text
            recom = comment.find_element_by_class_name('u_cbox_cnt_recomm').text
            unrecom = comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text
            print(f'댓글 #{comments_idx+1}:', [contents, name, date, recom, unrecom, news_url])
            comments_df.loc[comments_idx] = [contents, name, date, recom, unrecom, news_url]
            comments_idx += 1
        except NoSuchElementException:
            print("[삭제,부적절]")
            continue
            
    return comments_df

    
     

# scraping() : 스크래핑 함수

In [46]:
def scraping(news_max=3):
    wd = webdriver.Chrome('chromedriver')
    wd.implicitly_wait(3) # 처음 로딩 대기 시간
    
    wd.execute_script('window.open("about:blank", "_blank");') # 새로운 빈창을 뛰우는 자바스크립트
    tabs = wd.window_handles
    
    wd.switch_to.window(tabs[0])
    query = input("검색어 입력: ")
    search_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=' + query
    wd.get(search_url)
    
    news_idx = 0 
    news_df = pd.DataFrame(columns=("Title", "Press", "DateTime", "Article","Good", "Warm", "Sad", "Angry", "Want", "Recom"))
    comments_df = pd.DataFrame()
    
    while True:
        inline_list = wd.find_elements_by_class_name('info_group')
        for inline in inline_list:
            try:
                a_tag = inline.find_elements_by_tag_name('a')
                for a in a_tag:
                    hrefs = a.get_attribute('href')
                    print(hrefs)
                    
                    if hrefs.startswith('https://news.naver'):
                        news_url = hrefs
#                     else:
#                         pass
            except:
                continue
            
            wd.switch_to.window(tabs[1])
            

            wd.get(news_url)

            news_df.loc[news_idx] = news_scraping(news_url, wd)
            news_idx += 1

            df = comments_scraping(news_url, wd)
            comments_df = pd.concat([comments_df, df])
            
            if news_idx >=news_max:
                break
                
        if news_idx >=news_max:
            break
            
        try:
            wd.switch_to.window(tabs[0])
            wd.find_element_by_class_name('btn_next').click()
            time.sleep(2)
        
        except:
            break
    

    wd.close()
    
    return news_df, comments_df

In [47]:
news_df, comments_df = scraping()
# news_df

검색어 입력: 컴퓨터
http://www.etnews.com/
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002955338
뉴스: ['지테크시스템, 산업용 컴퓨터 선도한다', '전자신문', '2021.07.04. 오후 3:02', '0', '0', '0', '0', '0', '']
[댓글 스크래핑]https://zdnet.co.kr/
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002226467
뉴스: ['맥OS 빅서 실행하는 휴대용 미니컴퓨터 화제', 'ZDNet Korea', '2021.07.01. 오전 10:50', '9', '4', '0', '2', '0', '3']
[댓글더보기][답글][답글][댓글 스크래핑]댓글 #1: ['90년대 초*중반에 만들었으면, 큰 회사 스카우트 괬을껄....', 'wahi****', '2021.07.02. 09:42', '4', '0', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002226467']
댓글 #2: ['이미 2천년대 초반 태블릿피시 전신인 umpc가 있었음', 'fear****', '2021.07.02. 10:44', '0', '0', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002226467']
댓글 #3: ['그땐 하드웨어가 안됐쥬', 'kubo****', '2021.07.02. 13:29', '0', '0', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002226467']
댓글 #4: ['굳이', 'proc****', '2021.07.02. 

In [48]:
news_df

,Title,Press,DateTime,Article,Good,Warm,Sad,Angry,Want,Recom
0,"지테크시스템, 산업용 컴퓨터 선도한다",전자신문,2021.07.04. 오후 3:02,최근 산업용 컴퓨터 분야에서 시선을 한 몸에 받는 기업이 있다. 지난 16년간 산업...,0,0,0,0,0,
1,맥OS 빅서 실행하는 휴대용 미니컴퓨터 화제,ZDNet Korea,2021.07.01. 오전 10:50,“맥북을 주머니에 넣고 다니다가 필요할 때 꺼내 쓰면 얼마나 편리할까?”맥 운영체제...,9,4,0,2,0,3
2,맥OS 빅서 실행하는 휴대용 미니컴퓨터 화제,ZDNet Korea,2021.07.01. 오전 10:50,“맥북을 주머니에 넣고 다니다가 필요할 때 꺼내 쓰면 얼마나 편리할까?”맥 운영체제...,9,4,0,2,0,3


In [49]:
comments_df


,Contents,Name,Datetime,Recommend,Unrecommend,URL
0,"90년대 초*중반에 만들었으면, 큰 회사 스카우트 괬을껄....",wahi****,2021.07.02. 09:42,4,0,https://news.naver.com/main/read.nhn?mode=LSD&...
1,이미 2천년대 초반 태블릿피시 전신인 umpc가 있었음,fear****,2021.07.02. 10:44,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
2,그땐 하드웨어가 안됐쥬,kubo****,2021.07.02. 13:29,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
3,굳이,proc****,2021.07.02. 09:10,2,2,https://news.naver.com/main/read.nhn?mode=LSD&...
4,불법 해킨토시 신고합니다..,shin****,2021.07.02. 16:32,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
5,요즘 휴대폰도 램 16기가 나오는 판국에 8기가라니.,yunr****,2021.07.02. 15:06,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
6,역시 덕중의 덕은 양덕.. 대단하다...,medd****,2021.07.02. 09:42,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
7,난 니가 뭐라그러는지 1도 모르겠다?,akvl****,2021.07.02. 07:17,2,4,https://news.naver.com/main/read.nhn?mode=LSD&...
8,"우리는 이미 그것을 아이폰, 아이패드라고 부르기로 했습니다",itsf****,2021.07.02. 13:42,0,2,https://news.naver.com/main/read.nhn?mode=LSD&...
9,"아이패드 놔두고, 이걸 왜??",supe****,2021.07.02. 10:00,0,2,https://news.naver.com/main/read.nhn?mode=LSD&...
